In [5]:
import ngsolve as ngs
from do import solve, solve_2d
from ngsolve.webgui import Draw
import matplotlib.pyplot as plt
from copy import deepcopy
import netgen.occ as occ

levels = 5
deg = 5

######################################################
from bhdata import BHCurves
fun_dw  = BHCurves(4)
fun_w   = fun_dw.Integrate()
fun_ddw = fun_dw.Differentiate()

mu0 = 1.256636e-6
nu0 = 1/mu0

linear = "coil_plus|coil_minus"
nonlinear = "stator"

strom = 1e5
amps = 0.025**2*3.15*strom
print(f'Applying a current of {amps:.2f} Amps')
######################################################

r_coil = 0.025
y_coil = 0.05
r_outer = 0.1

c1 = occ.WorkPlane().Circle(r_outer).Face()
c2 = occ.WorkPlane().MoveTo(0,y_coil).Circle(r_coil).Face()
c3 = occ.WorkPlane().MoveTo(0,-y_coil).Circle(r_coil).Face()

full = occ.Glue([c1,c2,c3])

full.faces[0].name = 'stator'
full.faces[1].name = 'coil_plus'
full.faces[2].name = 'coil_minus'

full.edges[0].name = 'outer'

geoOCC = occ.OCCGeometry(full, dim = 2)

BH data from TEAM 13 problem
Applying a current of 196.88 Amps


In [6]:
meshes = []; Bs = []; Bps = []; its = []; errors = []

for i in range(levels):
    ngmesh = geoOCC.GenerateMesh()
    mesh = ngs.Mesh(ngmesh)

    for j in range(i):
        mesh.Refine()
    mesh.Curve(3)

    with ngs.TaskManager(): J = mesh.MaterialCF({'coil_plus': strom, 'coil_minus': -strom, 'stator': 0}, default = 0)
    H1 = ngs.H1(mesh, order = deg, dirichlet = "outer", autoupdate = True)
    L2 = ngs.L2(mesh, dim = 2, order = deg-1, autoupdate = True)
    A0 = ngs.GridFunction(H1)
    A0, it = solve_2d(H1, A0, mesh, deg, J, fun_w, fun_dw, fun_ddw)
    its.append(it)

    B = ngs.GridFunction(L2, nested = True)
    with ngs.TaskManager(): B.Set(ngs.grad(A0))
    Bs.append(B)

    if i>0:
        L2 = ngs.L2(mesh, dim = 2, order = deg-1)
        Bp = ngs.GridFunction(L2)
        with ngs.TaskManager(): Bp.Set(Bs[i-1])
        Bps.append(Bp)
        with ngs.TaskManager(): error = ngs.Integrate((B-Bp)**2, mesh)*(1/2)/ngs.Integrate((B)**2, mesh)*(1/2)
        print(error)
        errors.append(error)
        

    # ######################################################
    # # Refine mesh, apply prolongation
    # ######################################################

    # ngmesh2 = geoOCC.GenerateMesh()
    # mesh2 = ngs.Mesh(ngmesh2)

    # for j in range(i+1):
    #     mesh2.Refine()
    # mesh2.Curve()

    # V3L2 = ngs.L2(mesh2, dim=3, order = deg-1)
    # Bp = ngs.GridFunction(V3L2)
    # Bp.Set(B)
    # Bps.append(B)


print(its)


Using 2D mesh with ne= 197 elements and nv= 110 points and  2516 DOFs.
 
Using 2D mesh with ne= 788 elements and nv= 416 points and  9956 DOFs.
 
1.3980856701526355e-06
Using 2D mesh with ne= 3152 elements and nv= 1619 points and  39611 DOFs.
 
1.9920032552547145e-07
Using 2D mesh with ne= 12608 elements and nv= 6389 points and  158021 DOFs.
 
1.1360250704736986e-08
Using 2D mesh with ne= 50432 elements and nv= 25385 points and  631241 DOFs.
 


In [ ]:
import numpy as np
a = np.array(errors)
print(np.log2(a[:-1]/a[1:]))

[2.28707753 4.29844636 4.84169275]


In [ ]:
# with ngs.TaskManager(): Draw(B, mesh)